# Search behavior on Commons vs English Wikipedia

In [1]:
import pandas as pd
import numpy as np

from wmfdata import hive, mariadb

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [2]:
# query testsatisfaction2 dataset to obtain Commons and English Wikipedia search data.

## For now, it appears that something happened on Dec 10, 2019 to change either the data that
## was returned, or data gathering ended. More investigation is needed.

query = '''
SELECT
        dt,
        wiki,
        event.uniqueId AS event_id,
        event.pageViewId AS page_id,
        event.articleId AS article_id,
        event.searchSessionId AS session_id,
        MD5(LOWER(TRIM(event.query))) AS query_hash,
        event.action AS event_action,
        CASE
        WHEN event.position < 0 THEN NULL
        ELSE event.position
        END AS event_position,
        CASE
        WHEN event.action = "searchResultPage" AND event.hitsReturned > 0 THEN true
        WHEN event.action = "searchResultPage" AND event.hitsReturned IS NULL THEN false
        ELSE NULL
        END AS `some same-wiki results`,
        CASE
        WHEN event.action = "searchResultPage" AND event.hitsReturned > -1 THEN event.hitsReturned
        WHEN event.action = "searchResultPage" AND event.hitsReturned IS NULL THEN 0
        ELSE NULL
        END AS n_results,
        event.scroll,
        event.checkin,
        event.extraParams,
        event.msToDisplayResults AS load_time
FROM event.testsearchsatisfaction2
WHERE year IN (2019, 2020)
AND dt < "2020-02-03"
AND wiki IN ("commonswiki", "enwiki")
AND useragent.is_bot = false
AND event.source = "fulltext"
AND event.subTest IS NULL
AND CASE WHEN event.action = "searchResultPage" THEN event.msToDisplayResults IS NOT NULL
    WHEN event.action IN ("click", "iwclick", "ssclick")
        THEN event.position IS NOT NULL AND event.position > -1
    WHEN event.action = "visitPage" THEN event.pageViewId IS NOT NULL
    WHEN event.action = "checkin" THEN event.checkin IS NOT NULL AND event.pageViewId IS NOT NULL
    ELSE true
    END'''

In [3]:
fulltext_events_raw = hive.run(query)

In [ ]:
fulltext_events_raw.head()

In [5]:
fulltext_events_raw.dtypes

dt                         object
wiki                       object
event_id                   object
page_id                    object
article_id                float64
session_id                 object
query_hash                 object
event_action               object
event_position            float64
some same-wiki results     object
n_results                 float64
scroll                       bool
checkin                   float64
extraParams                object
load_time                 float64
user_agent                 object
dtype: object

In [6]:
## Drop the user agent column, it's not used in the subsequent analysis
## fulltext_events_raw.drop(columns = 'user_agent', inplace = True)

In [7]:
## Save the raw data
fulltext_events_raw.to_csv("data/fulltext_events_raw_enwiki_commons_Jan2020.csv")

In [4]:
## fulltext_events_raw = pd.read_csv("data/fulltext_events_raw_enwiki_commons_Jan2020.csv",
##                                 index_col = 0)